In [ ]:
import pandas as pd
import numpy as np
import scipy.signal as signal
import os
import math
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

sr = 200

print(f"CWD: {os.getcwd()}")

## Create list of participants with metadata

In [ ]:
# The times in minutes of the sync cue for each round, in their respective files
round_start_times = {
    "A" : 1,
    "B" : 5,
    "C" : 6,
    "D" : 24,
    "E" : 52,
    "F" : 70,
    "G" : 82,
    "H" : 93,
    "I" : 108
}

# Which locations are active per round
active_per_round = {
    "A" : list(range(1, 9)),
    "B" : list(range(1, 8)),
    "C" : list(range(1, 9)),
    "D" : list(range(1, 7)),
    "E" : list(range(1, 7)),
    "F" : list(range(1, 5)),
    "G" : list(range(1, 4)),
    "H" : list(range(1, 13)),
    "I" : list(range(1, 3))
}

In [ ]:
def get_participant_code(rnd: str, loc: int) -> str:
    return f"{rnd}{loc}"

In [ ]:
qom_dir = "./qom_std_data/"

groups = ["school", "staff", "public"]

participant_data_cols = ["round", "location", "participant_code", "source_qom_filepath", "group", "active", "sync_cue_mins"]

participants_df = pd.DataFrame(columns=participant_data_cols)

for group in groups:
    group_dir = os.path.join(qom_dir, group)

    csv_filenames = [file for file in os.listdir(group_dir) if file.endswith(".csv")]

    for csv_filename in csv_filenames:
        csv_filepath = os.path.join(group_dir, csv_filename)
        
        file_metadata = csv_filename.split("_")
        location = int(file_metadata[0])
        rnds = [*file_metadata[1]]
        sensor_id = int(file_metadata[2].split(".")[0])

        for rnd in rnds:
            idx = len(participants_df.index)
            participants_df.loc[idx, "round"] = rnd
            participants_df.loc[idx, "location"] = location
            participants_df.loc[idx, "participant_code"] = get_participant_code(rnd, location)
            participants_df.loc[idx, "source_qom_filepath"] = csv_filepath
            participants_df.loc[idx, "group"] = group
            participants_df.loc[idx, "active"] = True if location in active_per_round[rnd] else False
            participants_df.loc[idx, "sync_cue_mins"] = round_start_times[rnd]

participants_df.sort_values(["round", "location"], ascending=[True, True], axis=0, inplace=True)
participants_df.reset_index(drop=True, inplace=True)
participants_df

## Find round window indices for each participant

In [ ]:
def mins_to_samples(mins: float, sr: int) -> int:
    return int(mins* sr*60)

In [ ]:
round_len_mins = 6 # The length of each round in minutes after the sync cue
cue_window_size_mins = 8

start_tolerance = 0.5
stop_tolerance = 0.25

SAMP_PER_MIN = sr*60
ROUND_LEN_SAMPLES = int(round_len_mins * SAMP_PER_MIN) # Number of samples i.e., rows per round
CUE_WINDOW_SIZE_SAMPLES = int(cue_window_size_mins * SAMP_PER_MIN)

In [ ]:
participant_windows_df = participants_df.copy()

for i in range(len(participants_df.index)):
    
    row = participants_df.iloc[i]
    sync_cue_idx = mins_to_samples(row["sync_cue_mins"], sr) # Approximate location of sync cue in minutes
    n_attempts = 0 # No of attempts to find sync cue for a participant

    # Bypass unused round/location combinations
    if row["active"] == True:

        accepted = False

        # Loop until the user accepts a sync cue location and corresponding round bounds
        while not accepted:

            user_response = None
            sync_cue_idx_change = None
            
            n_attempts += 1

            if n_attempts > 1:
                print(f"Updated sync cue index to {sync_cue_idx} for attempt {n_attempts}.")
            
            # Get row data from participants dataframe
            filepath = row["source_qom_filepath"]
            
            # Load QoM data from CSV
            qom_data = pd.read_csv(filepath)
            qom_idx_len = len(qom_data.index)
            
            # Find sync cue window start and end points
            
            sync_cue_window_start = int(sync_cue_idx - mins_to_samples(cue_window_size_mins, sr) * 0.5)
            sync_cue_window_start = 0 if sync_cue_window_start < 0 else sync_cue_window_start
            sync_cue_window_stop = int(sync_cue_idx + mins_to_samples(cue_window_size_mins, sr) * 0.5)
            sync_cue_window_stop = qom_idx_len-1 if sync_cue_window_stop >= qom_idx_len else sync_cue_window_stop
    
            proposed_round_window_start = int(sync_cue_idx + mins_to_samples(start_tolerance, sr))
            proposed_round_window_start = 0 if proposed_round_window_start < 0 else proposed_round_window_start
            proposed_round_window_stop = int(sync_cue_idx + ROUND_LEN_SAMPLES - mins_to_samples(stop_tolerance, sr))
            proposed_round_window_stop = qom_idx_len-1 if proposed_round_window_stop >= qom_idx_len else proposed_round_window_stop
            
            # Plot QoM in sync cue window
            fig, axs = plt.subplot_mosaic([["top", "top"], ["left", "right"]], figsize=(12,5))

            axs["top"].plot(qom_data["qom"], lw=0.25, c="purple")
            axs["top"].axvline(sync_cue_idx, ls="--", c="black")
            axs["top"].axvline(proposed_round_window_start, ls="--", c="red")
            axs["top"].axvline(proposed_round_window_stop, ls="--", c="red")
            axs["top"].set_title(f"Whole file QoM: {filepath}")
            axs["top"].set_xlim(0, qom_idx_len)
            axs["top"].set_ylim(0, 2)
            axs["top"].grid(which="both", color="lightgrey", alpha=0.5)
            
            axs["left"].plot(qom_data["qom"][sync_cue_window_start:sync_cue_window_stop], lw=0.25, c="blue")
            axs["left"].axvline(sync_cue_idx, ls="--", c="black")
            axs["left"].set_title(f"{row['participant_code']} sync cue window QoM - proposed sync cue index: {sync_cue_idx}")
            axs["left"].set_xlim(sync_cue_window_start, sync_cue_window_stop)
            axs["left"].set_ylim(0, 2)
            axs["left"].grid(which="both", color="lightgrey", alpha=0.5)
    
            axs["right"].plot(qom_data["qom"][sync_cue_window_start:proposed_round_window_stop+mins_to_samples(1, sr)], lw=0.25, c="black")
            axs["right"].axvline(proposed_round_window_start, ls="--", c="red")
            axs["right"].axvline(proposed_round_window_stop, ls="--", c="red")
            axs["right"].set_title(f"{row['participant_code']} proposed round window ({proposed_round_window_start} - {proposed_round_window_stop})")
            axs["right"].set_xlim(sync_cue_window_start, proposed_round_window_stop+mins_to_samples(1, sr))
            axs["right"].set_ylim(0, 2)
            axs["right"].grid(which="both", color="lightgrey", alpha=0.5)

            plt.tight_layout()
            plt.show()

            time.sleep(0.5)
            
            user_response = input("Is the sync cue in the correct location? Y/N ")

            if user_response.lower()[0] == "y":
                participant_windows_df.loc[i, "round_window_start"] = int(proposed_round_window_start)
                participant_windows_df.loc[i, "round_window_stop"] = int(proposed_round_window_stop)
                print(f"{row['participant_code']}: accepted round window with bounds {proposed_round_window_start} to {proposed_round_window_stop}.")
                accepted = True
            
            elif user_response.lower()[0] == "n": 
                sync_cue_idx_change = None
                print(f"{row['participant_code']}: NOT accepted round window with bounds {proposed_round_window_start} to {proposed_round_window_stop}.")
                while type(sync_cue_idx_change) != int:
                    sync_cue_idx_change = input(f"Current sync cue index: {sync_cue_idx}\nEnter the +- change to the sync cue index: ")
                    try:
                        # Update value of sync cue idx for next attempt
                        sync_cue_idx_change = int(sync_cue_idx_change)
                        sync_cue_idx += sync_cue_idx_change
                        sync_cue_idx = 0 if sync_cue_idx < 0 else sync_cue_idx
                        sync_cue_idx = qom_idx_len-1 if sync_cue_idx >= qom_idx_len else sync_cue_idx
                        print(f"Updated sync cue index to {sync_cue_idx}, re-generating plots...")
                    except ValueError:
                        print(f"Enter something that can be converted to an int! Could not convert {sync_cue_idx_change} to an int.")
            else:
                print("Invalid response, enter Y/N.")

            # time.sleep(0.5)
            clear_output(wait=True)

    # else:
    #     print(f"Skipping {row['participant_code']} as this round/position location was not marked as active.")
    #     time.sleep(0.5)
    #     clear_output(wait=True)
        

print("Sync cues located for all known participants.")
print("Printing participants dataframe with results...")
participant_windows_df

In [ ]:
participant_windows_df

## Calculate scores for each participant

In [ ]:
def get_jerk(data_col: pd.Series) -> pd.Series:
    jerk = np.ediff1d(data_col)
    n_to_pad = np.abs(jerk.shape[0] - data_col.shape[0])
    jerk = np.pad(jerk, n_to_pad, mode="mean")
    print(data_col.shape, jerk.shape)
    return jerk[:-1]

g_to_mms2 = 9806.65 # For converting G (9.81m/s2) to mm/s2

In [ ]:
new_col_name = "score_mean_qom_jerk"

participant_windows_df = pd.read_csv("metadata/nm2024_extracted_participant_data.csv")

for i in range(len(participant_windows_df.index)):
    row = participant_windows_df.iloc[i]

    if row["active"] == True:
    
        round_start_idx = int(row["round_window_start"])
        round_stop_idx = int(row["round_window_stop"])
    
        participant_data = pd.read_csv(row["source_qom_filepath"])[round_start_idx:round_stop_idx]

        # Get absolute value of mean jerk in QoM across window
        score = np.abs(get_jerk(participant_data["qom"])).sum()
    
        participant_windows_df.loc[i, new_col_name] = score

scores_save_filename = "nm2024_extracted_participant_data.csv"
save_filepath = os.path.join("./metadata/", scores_save_filename)
participant_windows_df.to_csv(save_filepath, index=False)
participant_windows_df

## Plot scores as bar chart

In [ ]:
scores_df = participant_windows_df.copy()

save_filepath = "./figures/nm2024_scores_bar_chart.png"

podium_colours = ["gold", "silver", "peru"]

bar_width = 0.75

# Plot scores
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(16,9))

# Order participants by mean QoM
scores_df.sort_values("score_mean_qom", inplace=True, ascending=True)
scores_df.reset_index(drop=True, inplace=True)

for i, row in scores_df.iterrows():
    if row["active"] == True:
        if i < 3:
            color=podium_colours[i]
        else:
            color="crimson"
        ax1.bar(row["participant_code"], row["score_mean_qom"], color=color, ec="black", lw=0.5, zorder=3, width=bar_width)

ax1.set_xlabel("Participant code")
ax1.set_ylabel("Mean QoM")
ax1.set_title("Mean Quantity of Motion")
ax1.grid(axis="y", which="major", c="lightgrey", lw=1, zorder=1)
ax1.set_ylim(0.85, 1.15)

# Order participants by std
scores_df.sort_values("score_qom_std", inplace=True, ascending=True)
scores_df.reset_index(drop=True, inplace=True)

for i, row in scores_df.iterrows():
    if row["active"] == True:
        if i < 3:
            color=podium_colours[i]
        else:
            color="crimson"
        ax2.bar(row["participant_code"], row["score_qom_std"], color=color, ec="black", lw=0.5, zorder=3, width=bar_width)
    
ax2.set_xlabel("Participant code")
ax2.set_ylabel("Std of QoM")
ax2.set_title("Standard Deviation of QoM")
ax2.grid(axis="y", which="major", c="lightgrey", lw=1, zorder=1)
ax2.set_ylim(0, 0.012)

# Order participants by mean jerk
scores_df.sort_values("score_mean_qom_jerk", inplace=True, ascending=True)
scores_df.reset_index(drop=True, inplace=True)

for i, row in scores_df.iterrows():
    if row["active"] == True:
        if i < 3:
            color=podium_colours[i]
        else:
            color="crimson"
        ax3.bar(row["participant_code"], row["score_mean_qom_jerk"], color=color, ec="black", lw=0.5, zorder=3, width=bar_width)
    
ax3.set_xlabel("Participant code")
ax3.set_ylabel("Mean jerk")
ax3.set_title("Jerk (1st derivative) of QoM")
ax3.grid(axis="y", which="major", c="lightgrey", lw=1, zorder=1)
# ax3.set_ylim(0, 0.012)

plt.tight_layout()
# plt.savefig(save_filepath, dpi=600)
plt.show()

plt.figure(figsize=(18,6))

# Order participants by mean jerk
scores_df.sort_values("score_mean_qom_jerk", inplace=True, ascending=True)
scores_df.reset_index(drop=True, inplace=True)

n_participants = 0

for i, row in scores_df.iterrows():
    if row["active"] == True:
        n_participants += 1
        if i < 3:
            color=podium_colours[i]
        else:
            color="crimson"
        plt.bar(row["participant_code"], row["score_mean_qom_jerk"], color=color, ec="black", lw=0.5, zorder=3, width=bar_width)
    
plt.xlabel("Participant code")
plt.ylabel("Mean jerk")
plt.title("Norwegian Championship of Standstill 2024 - Scores")
plt.grid(axis="y", which="major", c="lightgrey", lw=1, zorder=1)
plt.ylim(200,350)
plt.xlim(-1, n_participants)
plt.xticks(rotation=45)
plt.savefig("./figures/nm2024_scores_mean_jerk.png", dpi=600)
plt.show()

In [ ]:
locations = np.sort(scores_df["location"].unique())

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(6,12))

for location in locations:
    loc_subset = scores_df[scores_df["location"] == location].copy()
    loc_subset.dropna(inplace=True)
    
    mean_mean_qom = loc_subset["score_mean_qom"].mean()
    mean_std = loc_subset["score_qom_std"].mean()
    mean_jerk = loc_subset["score_mean_qom_jerk"].mean()
    
    ax1.bar(location, mean_mean_qom, color="crimson", zorder=3)
    ax2.bar(location, mean_std, color="crimson", zorder=3)
    ax3.bar(location, mean_jerk, color="crimson", zorder=3)
    ax4.bar(location, len(loc_subset.index), color="darkblue", zorder=3)

ax1.set_xlabel("Location #")
ax1.set_ylabel("Mean QoM per location")
ax1.grid(axis="y", which="both", color="lightgrey", lw=1, zorder=1)
ax1.set_xticks(range(13))
ax1.set_ylim(0.8, 1.2)
ax1.set_title("QoM")

ax2.set_xlabel("Location #")
ax2.set_ylabel("Mean QoM std per location")
ax2.grid(axis="y", which="both", color="lightgrey", lw=1, zorder=1)
ax2.set_xticks(range(13))
ax2.set_title("QoM standard deviation")

ax3.set_xlabel("Location #")
ax3.set_ylabel("Mean QoM jerk per location")
ax3.grid(axis="y", which="both", color="lightgrey", lw=1, zorder=1)
ax3.set_xticks(range(13))
ax3.set_title("QoM jerk")

ax4.set_xlabel("Location #")
ax4.set_ylabel("# of rounds used")
ax4.grid(axis="y", which="both", color="lightgrey", lw=1, zorder=1)
ax4.set_xticks(range(13))
ax4.set_yticks(range(10))
ax4.set_title("# of Rounds per Location")

plt.tight_layout()
plt.savefig("./figures/nm2024_mean_scores_by_location.png", dpi=600, bbox_inches="tight")
plt.show()

## Create participant plots

In [ ]:
def butterworth_filter(data_to_filter: pd.Series, cutoff: float) -> pd.Series:
    sos = butter_filter = signal.butter(N=5, Wn=cutoff, btype="low", output="sos")
    filtered_data = signal.sosfilt(sos, data_to_filter)
    return filtered_data    

In [ ]:
for i in range(len(participant_windows_df.index)):
    row = participant_windows_df.iloc[i]

    if row["active"] == True:
    
        participant_code = row["participant_code"]
        
        round_start_idx = int(row["round_window_start"])
        round_stop_idx = int(row["round_window_stop"])
    
        participant_data = pd.read_csv(row["source_qom_filepath"])[round_start_idx:round_stop_idx].reset_index()
    
        round_len_samples = len(participant_data.index)
        round_len_seconds = round_len_samples / sr
        seconds = np.linspace(0, round_len_seconds, round_len_samples)
    
        y_upper = 0.01 # participant_data["qom"].max() + 0.25
        y_lower = 0 # participant_data["qom"].min() - 0.25
    
        jerk = np.abs(get_jerk(participant_data["qom"]))
        filtered_jerk = butterworth_filter(jerk, 0.01)
        
        # Create line of best fit
        m, b = np.polyfit(seconds, jerk, 1)
        
        fig, axs = plt.subplot_mosaic([["qom", "qom", "qom"], ["xy", "yz", "xz"]], figsize=(16,12))
    
        # Plot filtered jerk
        # axs["qom"].plot(seconds, jerk, lw=0.25, c="black", label="Jerk", alpha=0.25)
        axs["qom"].plot(seconds[10:], filtered_jerk[10:], lw=0.5, c="black", label="Jerk", alpha=1)
        axs["qom"].plot(seconds, m*seconds+b, lw=2, c="red", label="Linear regression")
        axs["qom"].set_ylim(y_lower, y_upper)
        axs["qom"].set_xlim(0,round_len_seconds)
        axs["qom"].grid(which="both", c="lightgrey", alpha=0.5)
        axs["qom"].set_title(f"{participant_code} jerk")
        axs["qom"].set_xlabel("Time (seconds)")
        axs["qom"].set_ylabel("Jerk (*9.81 m/s$^3$)")
    
        accel_data = pd.read_csv(row["source_accel_filepath"])[round_start_idx:round_stop_idx]
    
        x = butterworth_filter(accel_data["x"], 0.01)[1000:]
        y = butterworth_filter(accel_data["y"], 0.01)[1000:]
        z = butterworth_filter(accel_data["z"], 0.01)[1000:]
    
        x_mean = x.mean()
        y_mean = y.mean()
        z_mean = z.mean()
    
        x_std = (np.max(x) - np.min(x)) * 1.1
        y_std = (np.max(y) - np.min(y)) * 1.1
        z_std = (np.max(z) - np.min(z)) * 1.1
    
        lim_range = np.mean([x_std, y_std, z_std])
        
        axs["xy"].plot(x, y, lw=0.25, c="darkred")
        axs["xy"].set_title(f"{participant_code} acceleration in XY plane (Top view)")
        axs["xy"].set_xlabel("Acceleration in X (G)")
        axs["xy"].set_ylabel("Acceleration in Y (G)")
        axs["xy"].set_xlim(x_mean - lim_range, x_mean + lim_range)
        axs["xy"].set_ylim(y_mean - lim_range, y_mean + lim_range)
        
        axs["yz"].plot(y, z, lw=0.25, c="darkblue")
        axs["yz"].set_title(f"{participant_code} acceleration in YZ plane (Side view)")
        axs["yz"].set_xlabel("Acceleration in Y (G)")
        axs["yz"].set_ylabel("Acceleration in Z (G)")
        axs["yz"].set_xlim(y_mean - lim_range, y_mean + lim_range)
        axs["yz"].set_ylim(z_mean - lim_range, z_mean + lim_range)
        
        axs["xz"].plot(x, z, lw=0.25, c="darkgreen")
        axs["xz"].set_title(f"{participant_code} acceleration in XZ plane (Front view)")
        axs["xz"].set_xlabel("Acceleration in X (G)")
        axs["xz"].set_ylabel("Acceleration in Z (G)")
        axs["xz"].set_xlim(x_mean - lim_range, x_mean + lim_range)
        axs["xz"].set_ylim(z_mean - lim_range, z_mean + lim_range)
        
        # fig.legend()
        plt.tight_layout()
        plt.savefig(f"./figures/participant_plots/{participant_code}.pdf", dpi=600)
        plt.show()

In [ ]:
measurement = "std"

participants_df = pd.read_csv("./metadata/nm2024_extracted_participant_data.csv")

for i, row in participants_df.iterrows():
    qom_filepath = row["source_qom_filepath"].split("/")
    qom_filepath[1] = "accel_data"

    group_filename = qom_filepath[-1].split("\\")

    filename = group_filename[1].split(".")[0]
    group_filename[1] = f"{filename}_raw_acceleration.csv"
    
    new_filepath = os.path.join(*qom_filepath[:2], *group_filename)
    new_col_name = f"source_accel_filepath"
    participants_df.loc[i, new_col_name] = new_filepath
# participants_df.to_csv("./metadata/nm2024_extracted_participant_data.csv")
participants_df